In [104]:
import numpy as np

In [134]:
list1 = list(np.random.randint(1,10000,5000))
list2 = list1 + [list1[2]] * 5
list3 = list1.copy()
list3.append(-3)

lists = [list1,list2,list3]

In [124]:
def sort_list(long_list):
    new_list = []
    for element in long_list:
        new_list = sort_element(new_list, element)
    return new_list

def sort_element(mini_list, element, idx=0, how=None):

    if idx < 0:
        return [element] + mini_list
    
    elif idx == len(mini_list):
        return mini_list + [element]

    elif element < mini_list[idx]:
        if how == 'high':
            return mini_list[:idx] + [element] + mini_list[idx:]

        idx -= 1
        return sort_element(mini_list, element, idx, how='low')
    
    elif element == mini_list[idx]:
        return mini_list[:idx] + [element] + mini_list[idx:]
    
    elif element > mini_list[idx]:
        if how == 'low':
            return mini_list[:idx+1] + [element] + mini_list[idx+1:]

        idx += 1
        return sort_element(mini_list, element, idx, how='high')



In [125]:
def bubble_sort(nums):
    for _ in range(len(nums)-1):
        for i in range(len(nums)-1):

            if nums[i] > nums[i+1]:
                nums[i], nums[i+1] = nums[i+1], nums[i]
    
    return nums

In [126]:
def insertion_sort(nums):
    nums = list(nums)
    for i in range(len(nums)):
        current = nums.pop(i)
        j = i -1
        while j>=0 and nums[j]>current:
            j -= 1
        nums.insert(j+1, current)
    return nums

In [127]:
def merge_sort(nums):
    if len(nums) <= 1:
        return nums
    
    mid = len(nums) // 2

    left = nums[:mid]
    right = nums[mid:]

    left_sorted, right_sorted = merge_sort(left), merge_sort(right)

    sorted_nums = merge(left_sorted, right_sorted)
    return sorted_nums

def merge(nums1, nums2):

    merged = []
    i, j = 0, 0

    while i < len(nums1) and j < len(nums2):

        if nums1[i] <= nums2[j]:
            merged.append(nums1[i])
            i += 1
        else:
            merged.append(nums2[j])
            j += 1
        
    nums1_tail = nums1[i:]
    nums2_tail = nums2[j:]

    return merged + nums1_tail + nums2_tail

In [135]:
%%timeit

insertion_sort(list1)

564 ms ± 13.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [136]:
%%timeit

merge_sort(list1)

20.4 ms ± 238 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
